## KNN Item-Based

> ## 데이터 전처리 

### 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import math

movies_data = pd.read_csv('../data_movies.dat', sep='::', names=['movie_id', 'title', 'genres'], engine='python')
ratings_data = pd.read_csv('../data_ratings.dat', sep='::', names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

In [2]:
ratings_data

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
del ratings_data['timestamp']

In [4]:
ratings_data

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [5]:
df_ratings = ratings_data.set_index(['movie_id', 'user_id']).unstack()

In [6]:
df_ratings.rating

user_id,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movie_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,5.0,5.0,...,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
df_ratings2 = ratings_data.set_index(['user_id', 'movie_id']).unstack()
df_ratings2.rating

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
movie_ids = list(df_ratings.rating.index)

In [8]:
len(movie_ids)

3706

In [9]:
movie_cnt = df_ratings.shape[0]
user_cnt = df_ratings.shape[1]

### 유사도 행렬

In [10]:
sim_matrix_m = pd.DataFrame(np.zeros((movie_cnt, movie_cnt)))

In [11]:
sim_matrix_m = sim_matrix_m.reindex(movie_ids, columns=movie_ids, fill_value=0.0)

### 리스트로 바꾸기

In [12]:
def make_user_ids(df_row):
    df_row = df_row.dropna(axis=0)
    return set(df_row.index)

def make_ratings(df_row):
    df_row = df_row.dropna(axis=0)
    return dict((zip(df_row.index,df_row.values)))

In [13]:
ratings_result = pd.DataFrame()
ratings_result["movie_id"] = movie_ids
ratings_result = ratings_result.set_index("movie_id")
ratings_result["user_ids"] = df_ratings.rating.apply(lambda row : make_user_ids(row),axis=1)
ratings_result["user_ratings"] = df_ratings.rating.apply(lambda row : make_ratings(row),axis=1)
ratings_result = ratings_result.values

In [14]:
len(ratings_result)

3706

In [15]:
ratings_result[0]

array([{1, 6, 8, 9, 10, 18, 19, 21, 23, 26, 28, 34, 36, 38, 44, 45, 48, 49, 51, 56, 60, 65, 68, 73, 75, 76, 78, 80, 90, 92, 96, 99, 109, 112, 114, 117, 118, 119, 121, 123, 131, 132, 134, 136, 139, 142, 146, 147, 148, 149, 150, 151, 152, 156, 157, 162, 163, 168, 169, 173, 175, 181, 182, 184, 186, 187, 190, 193, 194, 195, 198, 202, 204, 213, 214, 215, 220, 223, 224, 225, 230, 231, 232, 236, 237, 239, 243, 246, 255, 258, 263, 264, 271, 272, 273, 284, 293, 294, 299, 300, 301, 302, 306, 307, 308, 310, 314, 321, 325, 326, 329, 333, 337, 338, 340, 343, 346, 350, 351, 352, 355, 366, 368, 369, 372, 376, 378, 380, 385, 386, 389, 392, 395, 402, 403, 411, 412, 413, 417, 418, 420, 424, 425, 428, 429, 434, 437, 438, 444, 451, 453, 456, 462, 463, 467, 474, 478, 479, 482, 490, 491, 495, 496, 499, 509, 516, 519, 523, 524, 528, 529, 531, 533, 536, 539, 541, 543, 546, 549, 550, 555, 556, 563, 566, 574, 575, 576, 577, 579, 583, 585, 588, 590, 595, 601, 605, 606, 610, 611, 613, 615, 620, 623, 624, 626, 628

> ## 유사도 

### 유사도 구하기

##### Cosine Similarity

In [16]:
%%time
from collections import deque

for i in range(movie_cnt-1):
    m_i = movie_ids[i]
    print(m_i)
    i_users = ratings_result[i][0]
    i_ratings = ratings_result[i][1]
    
    for j in range(i+1, movie_cnt):
        m_j = movie_ids[j]
        j_users = ratings_result[j][0]
        j_ratings = ratings_result[j][1]

        try:
            common_users = i_users & j_users

            if len(common_users) >= 10:
                i_s = deque()
                j_s = deque()
                for user_id in common_users:
                    i_s.append(i_ratings[user_id])
                    j_s.append(j_ratings[user_id])
                i_s = np.array(i_s)
                j_s = np.array(j_s)
                i_size = np.linalg.norm(i_s)
                j_size = np.linalg.norm(j_s)
                dot_ = np.dot(i_s, j_s)

                sim = dot_/((i_size)*(j_size))
                sim_matrix_m[m_i][m_j] = sim
                sim_matrix_m[m_j][m_i] = sim
        except:
            pass

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
110
111
112
113
114
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
28

2027
2028
2029
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2217
2218
2219
2221
2223
2226
2227
2231
2232
2233
2234
2235
2236


3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787
3788
3789
3790
3791
3792
3793
3794
3795
3796
3797
3798
3799
3800
3801
3802
3803
3804
3805
3806
3807
3808
3809
3810
3811
3812
3813
3814
3816
3817
3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
3885
3886
3887
3888
3889
3890
3891
3892
3893
3894
3895
3896
3897
3898
3899
3900
3901
3902
3903
3904
3905
3906
3908
3909
3910
3911
3912
3913
3914
3915
3916
3917
3918
3919
3920
3921
3922
3923
3924
3925
3926
3927
3928
3929
3930
3931
3932
3933


In [17]:
sim_matrix_m

,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,0.000000,0.948893,0.933795,0.946605,0.936290,0.953322,0.950446,0.939659,0.907479,0.955999,...,0.912694,0.0,0.812539,0.909188,0.943887,0.953194,0.963100,0.947089,0.963510,0.960953
2,0.948893,0.000000,0.922489,0.901500,0.948629,0.931789,0.942713,0.947975,0.934859,0.945344,...,0.897287,0.0,0.851829,0.897709,0.912506,0.934746,0.932032,0.957366,0.000000,0.925830
3,0.933795,0.922489,0.000000,0.913889,0.934831,0.916234,0.924982,0.947114,0.886145,0.935366,...,0.846017,0.0,0.792280,0.890559,0.841603,0.938587,0.912815,0.945197,0.000000,0.917308
4,0.946605,0.901500,0.913889,0.000000,0.928541,0.904156,0.929459,0.000000,0.760431,0.916696,...,0.000000,0.0,0.000000,0.000000,0.000000,0.914702,0.879885,0.000000,0.000000,0.945156
5,0.936290,0.948629,0.934831,0.928541,0.000000,0.921975,0.939886,0.962532,0.946057,0.946492,...,0.000000,0.0,0.939336,0.000000,0.000000,0.955236,0.913470,0.000000,0.000000,0.932621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.953194,0.934746,0.938587,0.914702,0.955236,0.947687,0.935887,0.949637,0.899511,0.951508,...,0.892779,0.0,0.819649,0.889485,0.887620,0.000000,0.933890,0.911029,0.917059,0.941560
3949,0.963100,0.932032,0.912815,0.879885,0.913470,0.945623,0.926119,0.878868,0.928032,0.945056,...,0.930297,0.0,0.816072,0.894305,0.964135,0.933890,0.000000,0.956075,0.935465,0.939272
3950,0.947089,0.957366,0.945197,0.000000,0.000000,0.930548,0.968086,0.000000,0.000000,0.975221,...,0.866025,0.0,0.000000,0.768558,0.000000,0.911029,0.956075,0.000000,0.000000,0.953424
3951,0.963510,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.936428,0.0,0.000000,0.000000,0.000000,0.917059,0.935465,0.000000,0.000000,0.959601


In [18]:
np.fill_diagonal(pd.DataFrame(sim_matrix_m).values, -100)

In [19]:
sim_matrix_m

,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,-100.000000,0.948893,0.933795,0.946605,0.936290,0.953322,0.950446,0.939659,0.907479,0.955999,...,0.912694,0.0,0.812539,0.909188,0.943887,0.953194,0.963100,0.947089,0.963510,0.960953
2,0.948893,-100.000000,0.922489,0.901500,0.948629,0.931789,0.942713,0.947975,0.934859,0.945344,...,0.897287,0.0,0.851829,0.897709,0.912506,0.934746,0.932032,0.957366,0.000000,0.925830
3,0.933795,0.922489,-100.000000,0.913889,0.934831,0.916234,0.924982,0.947114,0.886145,0.935366,...,0.846017,0.0,0.792280,0.890559,0.841603,0.938587,0.912815,0.945197,0.000000,0.917308
4,0.946605,0.901500,0.913889,-100.000000,0.928541,0.904156,0.929459,0.000000,0.760431,0.916696,...,0.000000,0.0,0.000000,0.000000,0.000000,0.914702,0.879885,0.000000,0.000000,0.945156
5,0.936290,0.948629,0.934831,0.928541,-100.000000,0.921975,0.939886,0.962532,0.946057,0.946492,...,0.000000,0.0,0.939336,0.000000,0.000000,0.955236,0.913470,0.000000,0.000000,0.932621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.953194,0.934746,0.938587,0.914702,0.955236,0.947687,0.935887,0.949637,0.899511,0.951508,...,0.892779,0.0,0.819649,0.889485,0.887620,-100.000000,0.933890,0.911029,0.917059,0.941560
3949,0.963100,0.932032,0.912815,0.879885,0.913470,0.945623,0.926119,0.878868,0.928032,0.945056,...,0.930297,0.0,0.816072,0.894305,0.964135,0.933890,-100.000000,0.956075,0.935465,0.939272
3950,0.947089,0.957366,0.945197,0.000000,0.000000,0.930548,0.968086,0.000000,0.000000,0.975221,...,0.866025,0.0,0.000000,0.768558,0.000000,0.911029,0.956075,-100.000000,0.000000,0.953424
3951,0.963510,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.936428,0.0,0.000000,0.000000,0.000000,0.917059,0.935465,0.000000,-100.000000,0.959601


In [20]:
sim_matrix_m.to_csv('cosine_smilarity_movie_10.csv', mode='w')

##### Pearson Similarity

In [ ]:
sim_matrix_m_p = pd.DataFrame(np.zeros((movie_cnt, movie_cnt)))

In [ ]:
sim_matrix_m_p = sim_matrix_m_p.reindex(movie_ids, columns=movie_ids, fill_value=0.0)

In [ ]:
sim_matrix_m_p

In [ ]:
%%time

from collections import deque

for i in range(movie_cnt-1):
    m_i = movie_ids[i]
    i_users = ratings_result[i][0]
    i_ratings = ratings_result[i][1]
    
    for j in range(i+1, movie_cnt):
        m_j = movie_ids[j]
        j_users = ratings_result[j][0]
        j_ratings = ratings_result[j][1]
        
        try:
            common_users = i_users & j_users

            if len(common_users) >= 10:
                i_s = deque()
                j_s = deque()
                for user_id in common_users:
                    i_s.append(i_ratings[user_id])
                    j_s.append(j_ratings[user_id])
                i_s = np.array(i_s)
                j_s = np.array(j_s)
                i_avg = np.mean(i_s)
                j_avg = np.mean(j_s)

                i_s = i_s - i_avg
                j_s = j_s - j_avg
                i_s_2 = np.sum(i_s ** 2)
                j_s_2 = np.sum(j_s ** 2)
                i_j_s = np.sum(i_s * j_s)
                
                sim = i_j_s / ((i_s_2 ** 0.5) * (j_s_2 ** 0.5))
                sim_matrix_m_p[m_i][m_j] = sim
                sim_matrix_m_p[m_j][m_i] = sim
        except:
            pass
                

In [ ]:
sim_matrix_m_p

> ## 예상 평점 구하기 - KNN 

### 준비물

In [32]:
movie_id_list = df_ratings.rating.index ## index to user_id
movie_id_to_idx = {}
for _ in range(len(movie_id_list)):
    movie_id_to_idx[movie_id_list[_]] = _

In [33]:
movie_id_list

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            3943, 3944, 3945, 3946, 3947, 3948, 3949, 3950, 3951, 3952],
           dtype='int64', name='movie_id', length=3706)

In [104]:
movie_id_to_idx

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 52: 50,
 53: 51,
 54: 52,
 55: 53,
 56: 54,
 57: 55,
 58: 56,
 59: 57,
 60: 58,
 61: 59,
 62: 60,
 63: 61,
 64: 62,
 65: 63,
 66: 64,
 67: 65,
 68: 66,
 69: 67,
 70: 68,
 71: 69,
 72: 70,
 73: 71,
 74: 72,
 75: 73,
 76: 74,
 77: 75,
 78: 76,
 79: 77,
 80: 78,
 81: 79,
 82: 80,
 83: 81,
 84: 82,
 85: 83,
 86: 84,
 87: 85,
 88: 86,
 89: 87,
 90: 88,
 92: 89,
 93: 90,
 94: 91,
 95: 92,
 96: 93,
 97: 94,
 98: 95,
 99: 96,
 100: 97,
 101: 98,
 102: 99,
 103: 100,
 104: 101,
 105: 102,
 106: 103,
 107: 104,
 108: 105,
 110: 106,
 111: 107,
 112: 108,
 113: 109,
 114: 

In [34]:
user_id_list = df_ratings.rating.columns.values ## index to movie_id
user_id_to_idx = {}
for _ in range(len(user_id_list)):
    user_id_to_idx[user_id_list[_]] = _

In [35]:
user_id_list

array([   1,    2,    3, ..., 6038, 6039, 6040], dtype=int64)

### KNN

In [37]:
sim_matrix_m

,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
1,-100.000000,0.948893,0.933795,0.946605,0.936290,0.953322,0.950446,0.939659,0.907479,0.955999,...,0.912694,0.0,0.812539,0.909188,0.943887,0.953194,0.963100,0.947089,0.963510,0.960953
2,0.948893,-100.000000,0.922489,0.901500,0.948629,0.931789,0.942713,0.947975,0.934859,0.945344,...,0.897287,0.0,0.851829,0.897709,0.912506,0.934746,0.932032,0.957366,0.000000,0.925830
3,0.933795,0.922489,-100.000000,0.913889,0.934831,0.916234,0.924982,0.947114,0.886145,0.935366,...,0.846017,0.0,0.792280,0.890559,0.841603,0.938587,0.912815,0.945197,0.000000,0.917308
4,0.946605,0.901500,0.913889,-100.000000,0.928541,0.904156,0.929459,0.000000,0.760431,0.916696,...,0.000000,0.0,0.000000,0.000000,0.000000,0.914702,0.879885,0.000000,0.000000,0.945156
5,0.936290,0.948629,0.934831,0.928541,-100.000000,0.921975,0.939886,0.962532,0.946057,0.946492,...,0.000000,0.0,0.939336,0.000000,0.000000,0.955236,0.913470,0.000000,0.000000,0.932621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,0.953194,0.934746,0.938587,0.914702,0.955236,0.947687,0.935887,0.949637,0.899511,0.951508,...,0.892779,0.0,0.819649,0.889485,0.887620,-100.000000,0.933890,0.911029,0.917059,0.941560
3949,0.963100,0.932032,0.912815,0.879885,0.913470,0.945623,0.926119,0.878868,0.928032,0.945056,...,0.930297,0.0,0.816072,0.894305,0.964135,0.933890,-100.000000,0.956075,0.935465,0.939272
3950,0.947089,0.957366,0.945197,0.000000,0.000000,0.930548,0.968086,0.000000,0.000000,0.975221,...,0.866025,0.0,0.000000,0.768558,0.000000,0.911029,0.956075,-100.000000,0.000000,0.953424
3951,0.963510,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.936428,0.0,0.000000,0.000000,0.000000,0.917059,0.935465,0.000000,-100.000000,0.959601


In [72]:
sim_matrix_m.values[movie_id_to_idx[52]]

array([0.95662163, 0.93189185, 0.92144496, ..., 0.93805106, 0.95279993,
       0.9649684 ])

In [105]:
sim_movies_list = pd.DataFrame()
sim_movies_list['movie_id'] = np.arange(len(movie_id_list)+1)
sim_movies_list['sim_movies_idxs'] = sim_matrix_m.apply(lambda row : np.array(row.sort_values(ascending=False).index), axis=1)
sim_movies_list
sim_movies_list = sim_movies_list.drop(0,axis=0)
sim_movies_list['movie_id'] = movie_id_list
sim_movies_list.set_index("movie_id")

,sim_movies_idxs
movie_id,
1,"[3114, 2873, 3171, 3222, 189, 3132, 2494, 2074..."
2,"[68, 3790, 2756, 295, 2593, 2205, 430, 3345, 3..."
3,"[3921, 68, 3450, 299, 963, 564, 1896, 937, 259..."
4,"[3090, 476, 2884, 1696, 3200, 3727, 2506, 1161..."
5,"[302, 869, 711, 2029, 1162, 1184, 3111, 2884, ..."
...,...
3948,"[3122, 1880, 3537, 211, 746, 602, 3574, 3077, ..."
3949,"[3158, 3586, 3118, 2983, 2237, 2744, 1896, 293..."
3950,"[394, 3158, 386, 1900, 210, 2842, 1507, 3122, ..."


In [113]:
sim_movies_list[sim_movies_list.movie_id==52]

,movie_id,sim_movies_idxs
51,52,NaN


In [91]:
sim_matrix_m.iloc[movie_id_to_idx[50]]

1       0.965937
2       0.942769
3       0.933722
4       0.941743
5       0.942471
          ...   
3948    0.958466
3949    0.960520
3950    0.959084
3951    0.967590
3952    0.967052
Name: 50, Length: 3706, dtype: float64

In [114]:
k = 50
movies_sims = {}
for movie_id in movie_id_list:
    print(movie_id)
    movies_sims[movie_id] = {}
    try:
        sim_movies = [ sim for sim in sim_movies_list[ sim_movies_list.movie_id==movie_id].values[0][1][:k] ]
        for sim_movie_id in sim_movies:
            movies_sims[movie_id][sim_movie_id] = sim_matrix_m.values[movie_id_to_idx[movie_id]][movie_id_to_idx[sim_movie_id]]
    except:
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
110
111
112
113
114
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
28

2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2217
2218
2219
2221
2223
2226
2227
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
2259
2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2271
2272
2273
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292


In [121]:
print(movies_sims[50])

{571: 0.9833638026873515, 649: 0.9805806756909202, 3338: 0.9831651298210415, 3854: 0.9832807616865793, 2029: 0.9784055691997853, 2577: 0.9811049102515927, 3859: 0.9834967446494399, 1044: 0.9784357531928178, 3613: 0.982288608505726, 2848: 0.9798297341371556, 1446: 0.9806468917940888, 3625: 0.9837442971777929, 298: 0.9844399218003572, 3756: 0.9817315188863662, 1198: 0.9786887344658037, 3634: 0.9824301462097466, 1844: 0.982942025036118, 309: 0.9799560029630435, 1617: 0.978168281574316, 568: 0.9787797262783878, 1759: 0.9787868076230676, 956: 0.9781333258735394, 1725: 0.9790852936135774, 2494: 0.981812155009845, 556: 0.9804048959528227, 2626: 0.9808889619917671, 3655: 0.9851159304629481, 3406: 0.9819002200360364, 335: 0.9787792315238957, 3537: 0.9885046978902512, 722: 0.9798958915919838, 3539: 0.9807573504567592, 3470: 0.9791346095068666, 214: 0.9834215720608249, 3801: 0.9786388500997794, 858: 0.9800801450607367, 3547: 0.9787951757282447, 2397: 0.9840187332606589, 3551: 0.9787733091924646, 

In [123]:
df_ratings2 = df_ratings2.fillna(-1)

In [124]:
movie_id = 1
user_id = 1
df_ratings2.rating[movie_id][user_id]

5.0

In [125]:
df_ratings3 = df_ratings2.values

In [126]:
user_idx = 6035
movie_idx = 3
df_ratings3[user_idx][movie_idx]

2.0

### 예상 평점 구하기

In [127]:
knn_result = {}

In [128]:
def make_prediction(row):
    knn_result[row.name] = {} ## 해당 유저의 안 본 영화 별 예상 평점을 저장할 딕셔너리
    print(row.name) ## row.name : 해당 행의 유저 아이디
    for movie, rating in row.iteritems():
        
        ## movie[1] 은 각 영화 아이디, rating은 뤠이팅
        if rating < 0: ## 해당 유저가 평가하지 않은 영화라면
            bunmo = 0
            bunja = 0
            for sim_movie, sim in movies_sims[movie[1]].items(): 
                if sim > 0:
                    sim_movie_rating = df_ratings3[user_id_to_idx[row.name]][movie_id_to_idx[sim_movie]] 
                    if sim_movie_rating > 0:
                        bunja += sim_movie_rating*sim
                        bunmo += sim
            if bunmo != 0:
                knn_result[row.name][movie[1]] = bunja/bunmo


In [130]:
%%time
df_ratings2.apply(lambda row: make_prediction(row), axis=1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340


user_id
1       None
2       None
3       None
4       None
5       None
        ... 
6036    None
6037    None
6038    None
6039    None
6040    None
Length: 6040, dtype: object

In [131]:
knn_result[1]

{11: 3.0000000000000004,
 17: 3.0,
 18: 2.9999999999999996,
 23: 3.0,
 27: 4.0,
 28: 4.0,
 29: 3.0,
 30: 3.4989867216309194,
 32: 3.0,
 35: 4.0,
 36: 3.0,
 38: 4.0,
 40: 4.000759464626621,
 43: 3.0,
 49: 4.3337235664759115,
 50: 3.0000000000000004,
 69: 4.243612805097028,
 73: 4.0,
 74: 5.0,
 76: 4.4975021269430435,
 78: 4.680252188640581,
 85: 4.798891447760049,
 88: 4.0,
 89: 3.0,
 94: 3.493099052125602,
 99: 3.5071363117839267,
 101: 4.6688554212517355,
 102: 4.0,
 105: 4.0,
 107: 2.9999999999999996,
 113: 5.0,
 117: 5.0,
 121: 3.0,
 122: 3.664346676408899,
 123: 4.3979821075727665,
 125: 4.668037308378484,
 128: 4.0,
 132: 4.0,
 135: 4.553171689081486,
 141: 4.0,
 151: 4.699845469376248,
 155: 5.0,
 160: 3.99852531425255,
 161: 4.497454495841283,
 164: 4.0,
 169: 3.0000000000000004,
 174: 4.0,
 176: 4.0,
 181: 3.0,
 183: 5.0,
 184: 4.324165964095698,
 185: 4.0,
 188: 4.0,
 194: 4.166615322287861,
 202: 4.594646498494191,
 204: 3.0,
 205: 4.333417432390885,
 206: 5.0,
 207: 4.251439

### 결과 저장

In [132]:
df_knn = pd.DataFrame()
df_knn["user_id"] = knn_result.keys()

In [133]:
def make_result(row):
    pre = deque()
    for movie_id, rating in knn_result[row.user_id].items():
        if rating > 0:
            pre.append((movie_id, rating))
    return sorted(list(pre), key=lambda x: x[1], reverse=True)

In [134]:
df_knn["predicts"] = df_knn.apply(lambda row: make_result(row), axis=1)

In [135]:
df_knn

,user_id,predicts
0,1,"[(2178, 5.000000000000001), (2191, 5.000000000..."
1,2,"[(140, 5.000000000000001), (172, 5.00000000000..."
2,3,"[(1208, 5.000000000000001), (269, 5.0000000000..."
3,4,"[(2350, 5.000000000000002), (3821, 5.000000000..."
4,5,"[(432, 5.000000000000001), (648, 5.00000000000..."
...,...,...
6035,6036,"[(197, 5.000000000000001), (1317, 5.0000000000..."
6036,6037,"[(7, 5.000000000000001), (485, 5.0000000000000..."
6037,6038,"[(750, 5.000000000000001), (2105, 5.0000000000..."
6038,6039,"[(105, 5.000000000000001), (322, 5.00000000000..."


In [136]:
pd.DataFrame(df_knn).to_csv("item_based_knn_predict_ratings_by_cosine.csv", mode='w')